# Data Preparation

## Packages

In [1]:
import os
import winsound
import numpy as np
import pandas as pd
import datetime as dt
from tqdm import tqdm

import math, collections
from scipy.stats import linregress

from matplotlib import pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_slope_s(y_axis):
    global window_s_ma
    x_axis = []
    for i in range(window_s_ma):
        x_axis.append(1 + ((i+1) * 0.0001 * 0.1))
    
    slope_tick, intercept, _, _, _ = linregress(x_axis, y_axis)
    slope_tick = math.degrees(math.atan(slope_tick))
    
    return(slope_tick)

def get_slope_l(y_axis):
    global window_l_ma
    x_axis = []
    for i in range(window_l_ma):
        x_axis.append(1 + ((i+1) * 0.0001 * 0.1))
    
    slope_tick, intercept, _, _, _ = linregress(x_axis, y_axis)
    slope_tick = math.degrees(math.atan(slope_tick))
    
    return(slope_tick)


def get_month(row):
    month_val = dt.datetime.strptime(row['TS'], '%Y-%m-%d %H:%M').date().month
    return(month_val)

def get_day(row):
    day_val = dt.datetime.strptime(row['TS'], '%Y-%m-%d %H:%M').date().day
    return(day_val)   

def get_hour(row):
    hour_val = dt.datetime.strptime(row['TS'], '%Y-%m-%d %H:%M').time().hour
    return(hour_val)   

def get_min(row):
    min_val = dt.datetime.strptime(row['TS'], '%Y-%m-%d %H:%M').time().minute
    return(min_val)   


def get_dow(row):
    dow = dt.datetime.strptime(df['TS'][0], '%Y-%m-%d %H:%M').weekday()
    return(dow)

In [3]:
def chunk_ticks(df, number_of_ticks):   
    global pip_diff
    
    df['tick'] = (df['Bid'] + df['Ask'])/2
    df = df[['tick']]

    temp_df = pd.DataFrame()
    open_list = []
    high_list = []
    low_list = []
    close_list = []
    mid_list = []


    for i in tqdm(range(0,len(df),number_of_ticks)):
        chunk_list = list(df['tick'][i:i+number_of_ticks])

        open_val = chunk_list[0]
        high_val = np.max(chunk_list)
        low_val = np.min(chunk_list)
        close_val = chunk_list[-1]

        open_list.append(open_val) 
        high_list.append(high_val)
        low_list.append(low_val)
        close_list.append(close_val) 
        mid_list.append((high_val + low_val)/2)
        

    temp_df['open'] = open_list
    temp_df['high'] = high_list
    temp_df['low'] = low_list    
    temp_df['close'] = close_list    
    temp_df['mid'] = mid_list    
    
    return(temp_df)

## Parameters

In [4]:
#pd.DataFrame(np.random.rand(14,4), columns=['a', 'b', 'c', 'd'])

year = 2019
source_file_path = f'data/tick_{year}.csv'

number_of_ticks = 60
pip_diff = 0.0001

diff_col = 'Close'

rsi_window = 10
rs_max = 1e6

window_s_ma = 10
window_l_ma = 50

## Read data

In [5]:
%%time
path, file_name = os.path.split(source_file_path)

target_file_name = 'tab_'+file_name
target_file_path = os.path.join(path, target_file_name)

df = pd.read_csv(source_file_path, nrows=10000)
#df = pd.read_csv(source_file_path)

Wall time: 12 ms


In [6]:
print(f'Total recs : {len(df)}')
df = chunk_ticks(df, number_of_ticks)
print(f'Reduced recs : {len(df)}')
df.head()

100%|██████████████████████████████████████████████████████████████████████████████| 167/167 [00:00<00:00, 8812.89it/s]

Total recs : 10000
Reduced recs : 167


,open,high,low,close,mid
0,1.14640,1.146700,1.146220,1.146700,1.146460
1,1.14670,1.146740,1.146630,1.146685,1.146685
2,1.14669,1.146770,1.146400,1.146435,1.146585
3,1.14644,1.146440,1.146215,1.146220,1.146328
4,1.14623,1.146415,1.146220,1.146260,1.146317


## Data manipulation

%%time
df['target_gain'] = abs(df['high'].shift(-1) - df['close'])
df['target_loss'] = abs(df['low'].shift(-1) - df['close'])

df['direction'] = 'same'
df.loc[(df['target_gain'] > df['target_loss']) & (df['target_gain'] > pip_diff), 'direction'] = 'increase'
df.loc[(df['target_loss'] > df['target_gain']) & (df['target_loss'] > pip_diff), 'direction'] = 'decrease'

df['diff'] = df['close'].diff()

df['gain'] = 0
df['loss'] = 0

df['gain'].loc[df['direction'] == 'increase'] = abs(df['target_gain'])
df['loss'].loc[df['direction'] == 'decrease'] = abs(df['target_loss'])

In [8]:
df['diff'] = df['mid'].diff()

df['gain'] = 0
df['loss'] = 0
df['gain'].loc[df['diff'] > 0] = abs(df['diff'])
df['loss'].loc[df['diff'] < 0] = abs(df['diff'])

df['avg_gain'] = df['gain'].rolling(window=rsi_window).mean()
df['avg_loss'] = df['loss'].rolling(window=rsi_window).mean()

df['rs'] = df['avg_gain']/df['avg_loss']
df['rs'] = df['rs'].where(df['rs'] <= rs_max, rs_max) 

df['rsi'] = 100 - (100 / (df['rs'] + 1))

#df['ssma'] = df['close'].rolling(window=window_s_ma).mean()
#df['lsma'] = df['close'].rolling(window=window_l_ma).mean()
#df['sma_diff'] = df['ssma'] - df['lsma']

#df['sema'] = df['close'].ewm(span=window_s_ma).mean()
#df['lema'] = df['close'].ewm(span=window_l_ma).mean()
#df['ema_diff'] = df['sema'] - df['lema']

# df['slope_s_c'] = df['close'].rolling(window=window_s_ma).apply(get_slope_s)
# df['slope_s_h'] = df['high'].rolling(window=window_s_ma).apply(get_slope_s)
# df['slope_s_l'] = df['low'].rolling(window=window_s_ma).apply(get_slope_s)
# df['slope_s_o'] = df['open'].rolling(window=window_s_ma).apply(get_slope_s)

#df['slope_l'] = df['close'].rolling(window=window_l_ma).apply(get_slope_l)
#df['sma_slope'] = df['ssma'].rolling(window=rsi_window).apply(get_slope)

df = df.dropna()
df = df.reset_index(drop=True)

## Write data to csv

In [ ]:
remove_columns = ['target_gain', 'target_loss']
df = df[df.columns.difference(remove_columns)]

df.to_csv(target_file_path, index = False)
winsound.PlaySound('C:\\Windows\\Media\\tada.wav', winsound.SND_ASYNC)

## Print Report

In [ ]:
['avg_gain', 
 'avg_loss', 
 'close', 
 'diff', 
 'direction', 
 'gain', 
 'high',
 'high_diff', 
 'loss', 
 'low', 
 'low_diff', 
 'open', 
 'rs', 
 'rsi']

In [ ]:
df.columns

In [ ]:
g= df['direction']
print(pd.concat([g.value_counts(), g.value_counts(normalize=True).mul(100)],axis=1, keys=('counts','percentage')))

df.head(10)

In [ ]:
df['avg'] = (df['high'] + df['low'])/2

In [9]:
df.head(10)

,open,high,low,close,mid,diff,gain,loss,avg_gain,avg_loss,rs,rsi
0,1.146335,1.146425,1.146315,1.146320,1.146370,0.000067,0.000067,0.000000,0.000038,0.000047,0.806452,44.642857
1,1.146315,1.146320,1.146170,1.146230,1.146245,-0.000125,0.000000,0.000125,0.000038,0.000059,0.635593,38.860104
2,1.146240,1.146405,1.146205,1.146390,1.146305,0.000060,0.000060,0.000000,0.000021,0.000059,0.355932,26.250000
3,1.146385,1.146405,1.146310,1.146320,1.146358,0.000053,0.000053,0.000000,0.000026,0.000049,0.535714,34.883721
4,1.146315,1.146535,1.146215,1.146315,1.146375,0.000017,0.000017,0.000000,0.000028,0.000023,1.204301,54.634146
5,1.146325,1.146350,1.146265,1.146325,1.146308,-0.000067,0.000000,0.000067,0.000028,0.000029,0.965517,49.122807
6,1.146310,1.146350,1.146290,1.146320,1.146320,0.000012,0.000012,0.000000,0.000029,0.000028,1.026316,50.649351
7,1.146340,1.146370,1.146270,1.146305,1.146320,0.000000,0.000000,0.000000,0.000029,0.000028,1.044643,51.091703
8,1.146260,1.146350,1.146250,1.146295,1.146300,-0.000020,0.000000,0.000020,0.000021,0.000030,0.700000,41.176471
9,1.146300,1.146355,1.146265,1.146340,1.146310,0.000010,0.000010,0.000000,0.000022,0.000021,1.035294,50.867052
